# Lab Assignment 2 - Part B: k-Nearest Neighbor Classification
Please refer to the `README.pdf` for full laboratory instructions.


## Problem Statement
In this part, you will implement the k-Nearest Neighbor (k-NN) classifier and evaluate it on two datasets:
- **Lenses Dataset**: A small dataset for contact lens prescription
- **Credit Approval (CA) Dataset**: Credit card application data with binary labels (+/-)

### Your Tasks
1. **Preprocess the data**: Handle missing values and normalize features
2. **Implement k-NN** with L2 distance
3. **Evaluate** on both datasets for different values of k
4. **Discuss** your results

### Datasets
The data files are located in the `credit 2017/` folder:
- `lenses.training`, `lenses.testing`
- `crx.data.training`, `crx.data.testing`
- `crx.names` (describes the features)


## Setup


In [1]:
# Library declarations
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter


In [2]:
# Data paths
DATA_PATH = "credit 2017/"

# Load Lenses data
def load_lenses_data():
    """Load the lenses dataset."""
    train_data = np.loadtxt(DATA_PATH + "lenses.training", delimiter=',')
    test_data = np.loadtxt(DATA_PATH + "lenses.testing", delimiter=',')
    
    # First column is ID, last column is label
    X_train = train_data[:, 1:-1]
    y_train = train_data[:, -1]
    X_test = test_data[:, 1:-1]
    y_test = test_data[:, -1]
    
    return X_train, y_train, X_test, y_test

# Load Credit Approval data
def load_credit_data():
    """
    Load the Credit Approval dataset.
    Note: This dataset contains missing values (?) and mixed types.
    You will need to preprocess it.
    """
    # TODO: Implement data loading
    # The data is comma-separated
    # Missing values are marked with '?'
    # Last column is the label ('+' or '-')
    pass

# Test loading lenses data
X_train_lenses, y_train_lenses, X_test_lenses, y_test_lenses = load_lenses_data()
print(f"Lenses - Train: {X_train_lenses.shape}, Test: {X_test_lenses.shape}")


Lenses - Train: (18, 3), Test: (6, 3)


## Task 1: Data Preprocessing
For the Credit Approval dataset, you need to:
1. **Handle missing values** (marked with '?'):
   - Categorical features: replace with mode/median
   - Numerical features: replace with label-conditioned mean
2. **Normalize features** using z-scaling:
   $$z_i^{(m)} = \frac{x_i^{(m)} - \mu_i}{\sigma_i}$$

Document exactly how you handle each feature!


In [11]:
def preprocess_credit_data(train_file, test_file):
    """
    Preprocess the Credit Approval dataset.
    
    Steps:
    1. Load the data
    2. Handle missing values
    3. Encode categorical variables
    4. Normalize numerical features
    
    Returns:
    --------
    X_train, y_train, X_test, y_test : numpy arrays
    """
    # TODO: Implement preprocessing
    # Hint: Read crx.names to understand the features
    # Feature types (from crx.names):
    # A1: categorical (b, a)
    # A2: continuous
    # A3: continuous
    # A4: categorical (u, y, l, t)
    # A5: categorical (g, p, gg)
    # A6: categorical (c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff)
    # A7: categorical (v, h, bb, j, n, z, dd, ff, o)
    # A8: continuous
    # A9: categorical (t, f)
    # A10: categorical (t, f)
    # A11: continuous
    # A12: categorical (t, f)
    # A13: categorical (g, p, s)
    # A14: continuous
    # A15: continuous
    # A16: label (+, -)

    # 1. Load the data
    # Which features are categorical vs continuous in 
    categorical_indices= [0, 3, 4, 5, 6, 8, 9, 11, 12]
    continuous_indices= [1, 2, 7, 10, 13, 14]
    
    #load the data into an two empty array of data and their labels
    train_data= []
    train_labels= []

    # open the training file, read it and sort it. I used the following website for function help on open(). Last column is the label and first 15 columns are the features: https://www.w3schools.com/python/ref_func_open.asp
    f= open(train_file, 'r')
    for line in f:
        fields= line.strip().split(',')
        train_labels.append(fields[-1])
        train_data.append(fields[:-1])
    f.close()
    
    #create empty array for the tested data
    test_data= []
    test_labels= []

    #read the tested data file and sort it
    f= open(test_file, 'r')
    for line in f:
        fields= line.strip().split(',')
        test_labels.append(fields[-1])
        test_data.append(fields[:-1])
    f.close()
    

    # 2. Handle missing values
    #dataset uses ? to mark a missing or unkown value, so we have to replace them in the arrays

    #loop through each feature column
    for feat_idx in range(15):
        #is there a ? in the training data?
        has_missing = any(row[feat_idx] == '?' for row in train_data)
        if not has_missing:
            # If there are no ?, skip the feature column
            continue
        
        #if the feature is categorical
        if feat_idx in categorical_indices:
            # find every value that is not ?
            values = [row[feat_idx] for row in train_data if row[feat_idx] != '?']
            mode_value = Counter(values).most_common(1)[0][0]
            #replace missing values with mode or most common
            for row in train_data:
                if row[feat_idx] == '?':
                    row[feat_idx] = mode_value
        
        #if the feature is continuous, replace the ? with the mean
        else:
            #find values where the label is + and not ?
            plus_values= [float(train_data[i][feat_idx]) 
                          for i in range(len(train_data)) 
                          if train_data[i][feat_idx] != '?' and train_labels[i] == '+']
            plus_mean= np.mean(plus_values)
            
            #same thing but for -
            minus_values = [float(train_data[i][feat_idx]) 
                           for i in range(len(train_data)) 
                           if train_data[i][feat_idx] != '?' and train_labels[i] == '-']
            minus_mean = np.mean(minus_values)
            
            #replace ? depending on the label for that row that the ? belongs in
            for i in range(len(train_data)):
                if train_data[i][feat_idx] == '?':
                    if train_labels[i] == '+':
                        train_data[i][feat_idx] = str(plus_mean)
                    else:
                        train_data[i][feat_idx] = str(minus_mean)
    
    # use the statistics from training data to fix it by looping by column
    for feat_idx in range(15):
        #check if there are missing values in the column
        has_missing= any(row[feat_idx] == '?' for row in test_data)
        if not has_missing:
            continue
        
        #see if the column contains letters
        if feat_idx in categorical_indices:
            # Use mode from training data
            values= [row[feat_idx] for row in train_data if row[feat_idx] != '?']
            #find mode so we can replace the missing values in the test data with it
            mode_value= Counter(values).most_common(1)[0][0]
            
            #replace missing values with the mode
            for row in test_data:
                if row[feat_idx] == '?':
                    row[feat_idx] = mode_value
        
        # for continous features
        else:
            # compute mean for +
            plus_values= [float(train_data[i][feat_idx]) 
                          for i in range(len(train_data)) 
                          if train_data[i][feat_idx] != '?' and train_labels[i] == '+']
            plus_mean= np.mean(plus_values)
            
            #now for -
            minus_values= [float(train_data[i][feat_idx]) 
                           for i in range(len(train_data)) 
                           if train_data[i][feat_idx] != '?' and train_labels[i] == '-']
            minus_mean= np.mean(minus_values)
            
            #replace missing values to set average value to whatever the label is
            for i in range(len(test_data)):
                if test_data[i][feat_idx] == '?':
                    if test_labels[i] == '+':
                        test_data[i][feat_idx] = str(plus_mean)
                    else:
                        test_data[i][feat_idx] = str(minus_mean)
    
    #3. Encode categorical variables
    
    #create encoders to store a mapping for each categorical feature. https://www.w3schools.com/python/ref_string_encode.asp
    encoders = {}
    for feat_idx in categorical_indices:
        #goes through every row and grabs the values in the columns, sorts alphabetically and removes duplicates.
        unique_values= sorted(set(row[feat_idx] for row in train_data))
        #creates like a dictionary
        encoders[feat_idx]= {val: i for i, val in enumerate(unique_values)}
    
    # encode training data
    for row in train_data:
        for feat_idx in categorical_indices:
            #take text value in the column, look it up in the encoder, and replace with the number
            row[feat_idx]= encoders[feat_idx][row[feat_idx]]
    
    #encode test data
    for row in test_data:
        for feat_idx in categorical_indices:
            if row[feat_idx] in encoders[feat_idx]:
                row[feat_idx] = encoders[feat_idx][row[feat_idx]]
            else:
                row[feat_idx] = len(encoders[feat_idx])
    
    #Convert to arrays
    X_train = np.array(train_data, dtype=float)
    X_test = np.array(test_data, dtype=float)
    
    # Convert + to 1 and - to 0
    y_train = np.array([1 if label == '+' else 0 for label in train_labels])
    y_test = np.array([1 if label == '+' else 0 for label in test_labels])
    
    #4. Normalize numerical features with next function
    X_train, X_test = z_normalize(X_train, X_test, continuous_indices)
    
    return X_train, y_train, X_test, y_test


def z_normalize(X_train, X_test, feature_indices):
    """
    Apply z-score normalization to specified features.
    
    Parameters:
    -----------
    X_train, X_test : numpy arrays
    feature_indices : list of indices for numerical features
    
    Returns:
    --------
    X_train_normalized, X_test_normalized : numpy arrays
    """
    # TODO: Implement z-normalization
    X_train_norm= X_train.copy()
    X_test_norm= X_test.copy()

    for feat_idx in feature_indices:
        #calculate mean and standard deviation from training data
        mean = np.mean(X_train[:, feat_idx])
        std = np.std(X_train[:, feat_idx])
        
        # make sure all numbers are real
        if std < 0:
            std = 1.0
        
        # Apply equation given above for training data and test
        X_train_norm[:, feat_idx]= (X_train[:, feat_idx] - mean)/ std
        X_test_norm[:, feat_idx]= (X_test[:, feat_idx] - mean)/ std
    
    return X_train_norm, X_test_norm


## Task 2: Implement k-NN Classifier
Implement k-NN with L2 (Euclidean) distance:
$$\mathcal{D}_{L2}(\mathbf{a}, \mathbf{b}) = \sqrt{\sum_i (a_i - b_i)^2}$$

For **categorical attributes**, use:
- Distance = 1 if values are different
- Distance = 0 if values are the same


In [ ]:
def l2_distance(a, b):
    """
    Compute L2 (Euclidean) distance between two vectors.
    
    Parameters:
    -----------
    a, b : numpy arrays of same shape
    
    Returns:
    --------
    distance : float
    """
    # TODO: Implement L2 distance
    # using formula above
    return np.sqrt(np.sum((a - b) ** 2))


def knn_predict(X_train, y_train, X_test, k):
    """
    Predict labels for test data using k-NN.
    
    Parameters:
    -----------
    X_train : numpy array of shape (n_train, n_features)
    y_train : numpy array of shape (n_train,)
    X_test : numpy array of shape (n_test, n_features)
    k : int, number of neighbors
    
    Returns:
    --------
    predictions : numpy array of shape (n_test,)
    """
    # TODO: Implement k-NN prediction
    # For each test sample:
    #   1. Compute distance to all training samples
    #   2. Find k nearest neighbors
    #   3. Predict using majority voting
    pass


def compute_accuracy(y_true, y_pred):
    """
    Compute classification accuracy.
    
    Returns:
    --------
    accuracy : float (between 0 and 1)
    """
    # TODO: Implement accuracy computation
    pass


In [7]:
"""
TASK 2: k-NN IMPLEMENTATION
Simple and straightforward implementations
"""

import numpy as np
from collections import Counter


def l2_distance(a, b):
    """
    Compute L2 (Euclidean) distance between two vectors.
    
    Parameters:
    -----------
    a, b : numpy arrays of same shape
    
    Returns:
    --------
    distance : float
    """
    # L2 distance formula: sqrt(sum((a_i - b_i)^2))
    return np.sqrt(np.sum((a - b) ** 2))


def knn_predict(X_train, y_train, X_test, k):
    """
    Predict labels for test data using k-NN.
    
    Parameters:
    -----------
    X_train : numpy array of shape (n_train, n_features)
    y_train : numpy array of shape (n_train,)
    X_test : numpy array of shape (n_test, n_features)
    k : int, number of neighbors
    
    Returns:
    --------
    predictions : numpy array of shape (n_test,)
    """
    predictions = []
    
    # For each test sample
    for test_sample in X_test:
        # Step 1: Compute distance to all training samples
        distances = []
        for i in range(len(X_train)):
            dist = l2_distance(test_sample, X_train[i])
            distances.append((dist, y_train[i]))
        
        # Step 2: Find k nearest neighbors
        # Sort by distance and take first k
        distances.sort(key=lambda x: x[0])
        k_nearest = distances[:k]
        
        # Step 3: Predict using majority voting
        # Get the labels of k nearest neighbors
        k_nearest_labels = [label for (dist, label) in k_nearest]
        
        # Find most common label
        most_common = Counter(k_nearest_labels).most_common(1)[0][0]
        predictions.append(most_common)
    
    return np.array(predictions)


def compute_accuracy(y_true, y_pred):
    """
    Compute classification accuracy.
    
    Returns:
    --------
    accuracy : float (between 0 and 1)
    """
    # Count how many predictions match the true labels
    correct = np.sum(y_true == y_pred)
    total = len(y_true)
    
    return correct / total


# =============================================================================
# EXAMPLE USAGE
# =============================================================================

if __name__ == "__main__":
    # Simple test
    X_train = np.array([[1, 2], [2, 3], [3, 4], [5, 6]])
    y_train = np.array([0, 0, 1, 1])
    X_test = np.array([[2, 2], [4, 5]])
    
    predictions = knn_predict(X_train, y_train, X_test, k=3)
    print(f"Predictions: {predictions}")
    
    # Test accuracy
    y_true = np.array([0, 1])
    acc = compute_accuracy(y_true, predictions)
    print(f"Accuracy: {acc:.4f}")

Predictions: [0 1]
Accuracy: 1.0000


## Task 3: Evaluate on Lenses Dataset
Test your k-NN implementation on the Lenses dataset for different values of k.


In [8]:
# TODO: Evaluate k-NN on Lenses dataset
# Try different values of k (e.g., 1, 3, 5, 7)

k_values = [1, 3, 5, 7]
lenses_results = []
# 
for k in k_values:
     predictions = knn_predict(X_train_lenses, y_train_lenses, X_test_lenses, k)
     accuracy = compute_accuracy(y_test_lenses, predictions)
     lenses_results.append((k, accuracy))
     print(f"k={k}: Accuracy = {accuracy:.4f}")


k=1: Accuracy = 1.0000
k=3: Accuracy = 1.0000
k=5: Accuracy = 1.0000
k=7: Accuracy = 1.0000


## Task 4: Evaluate on Credit Approval Dataset
First preprocess the data, then evaluate k-NN.


In [9]:
# TODO: Preprocess Credit Approval data
X_train_credit, y_train_credit, X_test_credit, y_test_credit = preprocess_credit_data(
     DATA_PATH + "crx.data.training",
     DATA_PATH + "crx.data.testing"
 )


In [10]:
# TODO: Evaluate k-NN on Credit Approval dataset
k_values = [1, 3, 5, 7]
credit_results = []
# 
for k in k_values:
    predictions = knn_predict(X_train_credit, y_train_credit, X_test_credit, k)
    accuracy = compute_accuracy(y_test_credit, predictions)
    credit_results.append((k, accuracy))
    print(f"k={k}: Accuracy = {accuracy:.4f}")


k=1: Accuracy = 0.7464
k=3: Accuracy = 0.7826
k=5: Accuracy = 0.8043
k=7: Accuracy = 0.7971


## Summary and Discussion

### Results Table

| Dataset | k=1 | k=3 | k=5 | k=7 |
|---------|-----|-----|-----|-----|
| Lenses | ? | ? | ? | ? |
| Credit Approval | ? | ? | ? | ? |

### Discussion
*Answer these questions:*
1. Which value of k works best for each dataset? Why do you think that is?
2. How did preprocessing affect your results on the Credit Approval dataset?
3. What are the trade-offs of using different values of k?
4. What did you learn from this exercise?
